In [1]:
import numpy as np
import pandas as pd

from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score, confusion_matrix
from sklearn.preprocessing import label_binarize

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

from art.attacks.evasion import SimBA, SpatialTransformation, DeepFool, BasicIterativeMethod, FastGradientMethod, ProjectedGradientDescent
from art.estimators.classification import PyTorchClassifier

import time

/opt/conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
head = {
            "model" : '',
            "attack_model": '',
            'epsilon': '',
            'Accuracy': '',
            'Macro Precision': '',
            'Weighted Precision': '',
            'Macro Recall': '',
            'Weighted Recall': '',
            'Macro F1': '',
            'Weighted F1': '',
            # 'Macro AUC': '',
            # 'Weighted AUC': '',
            'TPR': '',
            'FNR': '',
            'TNR': '',
            'FPR': '',
        }
head = pd.DataFrame([head])
head.to_csv("/home/jovyan/A2PM/preprocessor/GaussianNoise.csv", mode='a', index=False)




In [3]:
from art.defences.postprocessor.gaussian_noise import GaussianNoise

gaussian_noise = GaussianNoise()


In [4]:
def calculate_performance_metrics(X_test, y_true, model, model_name, attack_name, eps):
    model.eval()
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    
    all_preds = []
    all_labels = []
    probabilities = []

    num_classes = len(np.unique(y_true))
    
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
    y_test_tensor = torch.tensor(y_test, dtype=torch.long)
    test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
    test_loader = DataLoader(dataset=test_dataset)

    with torch.no_grad():
        
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)

            outputs_np = outputs.cpu().numpy()       
            outputs_noisy = gaussian_noise(outputs_np)            
            outputs_noisy_tensor = torch.tensor(outputs_noisy).to(device)
            
            preds = torch.argmax(outputs_noisy_tensor, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            probabilities.extend(torch.nn.functional.softmax(outputs, dim=1).cpu().numpy())
        
        all_preds = np.array(all_preds)
        all_labels = np.array(all_labels)
        probabilities = np.array(probabilities)
        
        accuracy = accuracy_score(all_labels, all_preds)

        precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(all_labels, all_preds, average='macro')
        precision_weighted, recall_weighted, f1_weighted, _ = precision_recall_fscore_support(all_labels, all_preds, average='weighted')
    
        # macro_auc = roc_auc_score(label_binarize(all_labels, classes=range(num_classes)), probabilities[:,1], average='macro')
        # weighted_auc = roc_auc_score(label_binarize(all_labels, classes=range(num_classes)), probabilities[:,1], average='weighted')

        cm = confusion_matrix(all_labels, all_preds)

        def calculate_class_metrics_macro(cm, class_index):
            TP = cm[class_index, class_index]
            FP = cm[:, class_index].sum() - TP
            FN = cm[class_index, :].sum() - TP
            TN = cm.sum() - (TP + FP + FN)
            
            TPR = TP / (TP + FN) if (TP + FN) != 0 else 0  
            TNR = TN / (TN + FP) if (TN + FP) != 0 else 0  
            FPR = FP / (FP + TN) if (FP + TN) != 0 else 0  
            FNR = FN / (FN + TP) if (FN + TP) != 0 else 0  
            
            return TPR, TNR, FPR, FNR
            
        metrics = np.array([calculate_class_metrics_macro(cm, i) for i in range(num_classes)])
        TPR_macro, TNR_macro, FPR_macro, FNR_macro = np.mean(metrics, axis=0)

        print(f"Accuracy: {accuracy}")
        
        print("\nmacro")
        print(f"Precision: {precision_macro}\nRecall: {recall_macro}\nF1 Score: {f1_macro}")
    
        print("\nweighted")
        print(f"Precision: {precision_weighted}\nRecall: {recall_weighted}\nF1 Score: {f1_weighted}")
        print()
        
        print(f"Mean FNR: {FNR_macro}\nMean TNR: {TNR_macro}\nMean FPR: {FPR_macro}\nMean TPR: {TPR_macro}")

        new_row = {
            "model" : model_name,
            "attack_model" : attack_name,
            'epsilon': eps,
            'Accuracy': accuracy,
            'Macro Precision': precision_macro,
            'Weighted Precision': precision_weighted,
            'Macro Recall': recall_macro,
            'Weighted Recall': recall_weighted,
            'Macro F1': f1_macro,
            'Weighted F1': f1_weighted,
            # 'Macro AUC': macro_auc,
            # 'Weighted AUC': weighted_auc,
            'TPR': TPR_macro,
            'FNR': FNR_macro,
            'TNR': TNR_macro,
            'FPR': FPR_macro,
        }
        new_row_df = pd.DataFrame([new_row])
        new_row_df.to_csv("/home/jovyan/A2PM/preprocessor/GaussianNoise.csv", mode='a', index=False, header=False)




In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")

Using cuda device


In [6]:
x_test = np.load('/home/jovyan/Wustl_iiot/x_test.npy')
x_train = np.load('/home/jovyan/Wustl_iiot/x_train.npy')
x_val = np.load('/home/jovyan/Wustl_iiot/x_val.npy')
y_test = np.load('/home/jovyan/Wustl_iiot/y_test.npy')
y_train = np.load('/home/jovyan/Wustl_iiot/y_train.npy')
y_val = np.load('/home/jovyan/Wustl_iiot/y_val.npy')

In [7]:
input_shape = x_train.shape[1]
output_shape = len(np.unique(y_train))

In [8]:
class DNNModel(nn.Module):
    def __init__(self, input_size, output_size):
        super(DNNModel, self).__init__()
        self.fc1 = nn.Linear(input_size, 50)
        self.fc2 = nn.Linear(50, 30)
        self.fc3 = nn.Linear(30, 20)
        self.fc4 = nn.Linear(20, output_size)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x

In [9]:
model = DNNModel(input_size=input_shape, output_size=output_shape).to(device)
model.load_state_dict(torch.load("/home/jovyan/Wustl_iiot/transfer_attack/dnn_pytorch.pt"))

<All keys matched successfully>

In [10]:
from art.attacks.evasion import SimBA, SpatialTransformation, DeepFool, BasicIterativeMethod, FastGradientMethod, ProjectedGradientDescent


In [10]:
calculate_performance_metrics(x_test, y_test, model, 'DNN', 'baseline', '0')


Accuracy: 0.9998869124157602

macro
Precision: 0.9230338228387467
Recall: 0.9512335965005867
F1 Score: 0.9313810979752055

weighted
Precision: 0.9999113268629177
Recall: 0.9998869124157602
F1 Score: 0.9998928870300008

Mean FNR: 0.048766403499413205
Mean TNR: 0.9999772604156446
Mean FPR: 2.2739584355390455e-05
Mean TPR: 0.9512335965005867


In [14]:
epsilon_values = [0.01, 0.1, 0.2, 0.3]

# Iterate over epsilon values
print("start BIM")
for epsilon in epsilon_values:
    filename = f'/home/jovyan/Wustl_iiot/transfer_attack/x_test_adv_BIM_eps_{epsilon}.npy'
    x_test_adv = np.load(filename)

    calculate_performance_metrics(x_test_adv, y_test, model, 'DNN', 'BIM', epsilon)

print("start FGSM")
for epsilon in epsilon_values:
    filename = f'/home/jovyan/Wustl_iiot/transfer_attack/x_test_adv_FGSM_eps_{epsilon}.npy'
    x_test_adv = np.load(filename)

    calculate_performance_metrics(x_test_adv, y_test, model, 'DNN', 'FGSM', epsilon)

print("start PGD")
for epsilon in epsilon_values:
    filename = f'/home/jovyan/Wustl_iiot/transfer_attack/x_test_adv_PGD_eps_{epsilon}.npy'
    x_test_adv = np.load(filename)

    calculate_performance_metrics(x_test_adv, y_test, model, 'DNN', 'PGD', epsilon)

print("start DF")
DF_eps = [0.01, 0.1, 0.2, 0.3]
for epsilon in DF_eps:
    filename = f'/home/jovyan/Wustl_iiot/transfer_attack/x_test_adv_DF_eps_{epsilon}.npy'
    x_test_adv = np.load(filename)

    calculate_performance_metrics(x_test_adv, y_test, model, 'DNN', 'DF', epsilon)

start BIM
Accuracy: 0.9993507934978827

macro
Precision: 0.8306303831905089
Recall: 0.9085009780142645
F1 Score: 0.85324369726962

weighted
Precision: 0.9995074877806105
Recall: 0.9993507934978827
F1 Score: 0.9994141313508288

Mean FNR: 0.09149902198573553
Mean TNR: 0.9998696059648842
Mean FPR: 0.00013039403511587954
Mean TPR: 0.9085009780142645
Accuracy: 0.0033591200948260335

macro
Precision: 0.00803864665832432
Recall: 0.026261247727427238
F1 Score: 0.0013771871067921075

weighted
Precision: 0.036385554259733235
Recall: 0.0033591200948260335
F1 Score: 0.004641048448180901

Mean FNR: 0.9737387522725728
Mean TNR: 0.6454381513682771
Mean FPR: 0.35456184863172285
Mean TPR: 0.026261247727427238
Accuracy: 0.00021360988134180514

macro
Precision: 4.5704453495718566e-05
Recall: 0.006189320388349515
F1 Score: 9.073885446644218e-05

weighted
Precision: 1.5773820509259402e-06
Recall: 0.00021360988134180514
F1 Score: 3.13163880999813e-06

Mean FNR: 0.9938106796116504
Mean TNR: 0.640390981934693

/home/jovyan/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/jovyan/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy: 0.0

macro
Precision: 0.0
Recall: 0.0
F1 Score: 0.0

weighted
Precision: 0.0
Recall: 0.0
F1 Score: 0.0

Mean FNR: 1.0
Mean TNR: 0.6397636476071518
Mean FPR: 0.36023635239284824
Mean TPR: 0.0
start FGSM
Accuracy: 0.9994764463692603

macro
Precision: 0.8449368661475398
Recall: 0.9139732565447953
F1 Score: 0.8733144172725581

weighted
Precision: 0.9995361873308365
Recall: 0.9994764463692603
F1 Score: 0.9994991930644395

Mean FNR: 0.08602674345520457
Mean TNR: 0.9998947634148785
Mean FPR: 0.00010523658512162927
Mean TPR: 0.9139732565447953
Accuracy: 0.00608159897467257

macro
Precision: 0.05032135628002497
Recall: 0.07245840543432319
F1 Score: 0.007808834753223966

weighted
Precision: 0.057312206861081
Recall: 0.00608159897467257
F1 Score: 0.006994386074575815

Mean FNR: 0.9275415945656768
Mean TNR: 0.6621920353274561
Mean FPR: 0.3378079646725438
Mean TPR: 0.07245840543432319
Accuracy: 0.0035894836923515095

macro
Precision: 0.011991789334299719
Recall: 0.05951396700857492
F1 Sco

/home/jovyan/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/jovyan/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy: 0.0

macro
Precision: 0.0
Recall: 0.0
F1 Score: 0.0

weighted
Precision: 0.0
Recall: 0.0
F1 Score: 0.0

Mean FNR: 1.0
Mean TNR: 0.6397744023364154
Mean FPR: 0.36022559766358453
Mean TPR: 0.0
start DF
Accuracy: 3.769586141325973e-05

macro
Precision: 0.22224128799915266
Recall: 0.034433137978654126
F1 Score: 0.05896048124397185

weighted
Precision: 0.00031176312737877425
Recall: 3.769586141325973e-05
F1 Score: 6.563333505114818e-05

Mean FNR: 0.965566862021346
Mean TNR: 0.6850647502806015
Mean FPR: 0.3149352497193985
Mean TPR: 0.034433137978654126
Accuracy: 3.350743236734198e-05

macro
Precision: 0.22274633850420317
Recall: 0.030586984132500283
F1 Score: 0.051618976001666726

weighted
Precision: 0.000315697712240091
Recall: 3.350743236734198e-05
F1 Score: 5.788642559123317e-05

Mean FNR: 0.9694130158674998
Mean TNR: 0.6850604685885905
Mean FPR: 0.3149395314114095
Mean TPR: 0.030586984132500283
Accuracy: 2.9319003321424233e-05

macro
Precision: 0.2000190639595844
Recall: 0.0267

In [12]:
epsilon_values = [0.01, 0.1, 0.2, 0.3]

# Iterate over epsilon values
print("start AutoPGD")
for epsilon in epsilon_values:
    filename = f'/home/jovyan/Wustl_iiot/transfer_attack/x_test_adv_AutoPGD_eps_{epsilon}.npy'
    x_test_adv = np.load(filename)

    calculate_performance_metrics(x_test_adv, y_test, model, 'DNN', 'AutoPGD', epsilon)

print("start BA")
for epsilon in epsilon_values:
    filename = f'/home/jovyan/Wustl_iiot/transfer_attack/x_test_adv_BA_eps_{epsilon}.npy'
    x_test_adv = np.load(filename)

    calculate_performance_metrics(x_test_adv, y_test, model, 'DNN', 'BA', epsilon)

start AutoPGD
Accuracy: 0.9989905885999338

macro
Precision: 0.7700972395681785
Recall: 0.8836999968871748
F1 Score: 0.7939956880858982

weighted
Precision: 0.9993592087198993
Recall: 0.9989905885999338
F1 Score: 0.9991138045966662

Mean FNR: 0.1163000031128252
Mean TNR: 0.9997709067222271
Mean FPR: 0.00022909327777280238
Mean TPR: 0.8836999968871748
Accuracy: 0.0024795499951833066

macro
Precision: 0.013519345113420612
Recall: 0.012070650379663256
F1 Score: 0.009209689187603856

weighted
Precision: 0.033087890316011476
Recall: 0.0024795499951833066
F1 Score: 0.004600508275147053

Mean FNR: 0.9879293496203367
Mean TNR: 0.6293381169106281
Mean FPR: 0.37066188308937187
Mean TPR: 0.012070650379663256
Accuracy: 0.00041046604649993927

macro
Precision: 0.005192252320166797
Recall: 0.011985665478976604
F1 Score: 0.00619111329523783

weighted
Precision: 0.0051381092676875875
Recall: 0.00041046604649993927
F1 Score: 0.000723646612808477

Mean FNR: 0.9880143345210234
Mean TNR: 0.620961287608096

In [14]:
filename = f'/home/jovyan/Wustl_iiot/transfer_attack/x_test_adv_HopSkipJump_eps_0.npy'
x_test_adv = np.load(filename)

calculate_performance_metrics(x_test_adv, y_test, model, 'DNN', 'HopSkipJump', '0')

Accuracy: 0.4224491420003099

macro
Precision: 0.20987350737724783
Recall: 0.2553126361943977
F1 Score: 0.1475712623498451

weighted
Precision: 0.889006194522042
Recall: 0.4224491420003099
F1 Score: 0.5666907763914174

Mean FNR: 0.7446873638056023
Mean TNR: 0.8306504365053312
Mean FPR: 0.16934956349466881
Mean TPR: 0.2553126361943977


In [11]:
print("start ZOO")
ZOO_epsilon_values = [0, 0.01, 0.1, 0.2, 0.3]
# Iterate over epsilon values
for epsilon in ZOO_epsilon_values:
    filename = f'/home/jovyan/Wustl_iiot/transfer_attack/x_test_adv_ZOO_eps_{epsilon}.npy'
    x_test_adv = np.load(filename)

    calculate_performance_metrics(x_test_adv, y_test, model, 'DNN', 'ZOO', epsilon)

start ZOO
Accuracy: 0.999082734038944

macro
Precision: 0.7654860900617877
Recall: 0.890094694422649
F1 Score: 0.7982447901928279

weighted
Precision: 0.9993771788338607
Recall: 0.999082734038944
F1 Score: 0.9991773135655936

Mean FNR: 0.10990530557735098
Mean TNR: 0.9997896907193645
Mean FPR: 0.00021030928063551008
Mean TPR: 0.890094694422649
Accuracy: 0.9991204299003573

macro
Precision: 0.7802282558792677
Recall: 0.9023613113009163
F1 Score: 0.8139727011815353

weighted
Precision: 0.9993802116871651
Recall: 0.9991204299003573
F1 Score: 0.9992012360798628

Mean FNR: 0.09763868869908361
Mean TNR: 0.9997970554198113
Mean FPR: 0.00020294458018868164
Mean TPR: 0.9023613113009163
Accuracy: 0.9990743571808521

macro
Precision: 0.7680662101701781
Recall: 0.8777618411917375
F1 Score: 0.7949064228440801

weighted
Precision: 0.9993766068187524
Recall: 0.9990743571808521
F1 Score: 0.9991748573971085

Mean FNR: 0.12223815880826246
Mean TNR: 0.9997879565705334
Mean FPR: 0.0002120434294665339
Mean